Good job last week! Your code looks good!

Now we need to create a detector that is able to decide which person's code to use to parse the html. 

In this notebook we are going to write code to determine whether to use the logic you developed last week to transform the an HTML file into a dataframe, or to use someone else's code. 

#### Let's Get Setup!

In the cell below re-direct the file path to your directory. 

In [13]:
html_content = r"C:\Users\abw13\Research\research_assistants copy\research_assistants copy\raw_html_files\meet_493916.html"

Read the html file you wrote code for last week.

In [17]:
from bs4 import BeautifulSoup
with open(html_content, 'r', encoding='utf-8') as f:
    html = f.read()

soup = BeautifulSoup(html, 'html.parser')
print(soup.prettify()[:]) 

<!DOCTYPE html>
<html lang="en" xmlns:="">
 <head>
  <script src="https://cmp.osano.com/AzyWAQS5NWEEWkU9/eab0a836-8bac-45b1-8b3e-e92e57e669db/osano.js?language=en">
  </script>
  <script src="https://www.flolive.tv/osano-flo.js">
  </script>
  <!-- Google Tag Manager -->
  <script>
   (function (w, d, s, l, i) {
            w[l] = w[l] || [];
            w[l].push({
                'gtm.start':
                    new Date().getTime(), event: 'gtm.js'
            });
            var f = d.getElementsByTagName(s)[0],
                j = d.createElement(s), dl = l != 'dataLayer' ? '&l=' + l : '';
            j.async = true;
            j.src =
                'https://www.googletagmanager.com/gtm.js?id=' + i + dl;
            f.parentNode.insertBefore(j, f);
        })(window, document, 'script', 'dataLayer', 'GTM-M962P37');
  </script>
  <!-- End Google Tag Manager -->
  <title>
   CVAA Preview 2022 - Results (Raw)
  </title>
  <!-- Meta -->
  <meta charset="utf-8"/>
  <meta content="te

### Step 1: Visually Compare the Webpages  

Before writing any code, **take a careful look** at the webpages to spot what makes *your* HTML different from your teammates’ HTML.  
These differences will later form the basis of the “logic” you write in your detector function.

**Here are the URLs we’ll compare:**
- [Aragon’s Center Meet 3 (2008)](https://ca.milesplit.com/meets/44115-aragons-center-meet-3-2008/results)  
- [CVAA Preview (2022)](https://ca.milesplit.com/meets/493916-cvaa-preview-2022/results/846055/raw)  
- [CVL Meet 3 (ACE, 2022)](https://ca.milesplit.com/meets/494231-cvl-meet-3-ace-2022/results/846020/raw)

**Here’s the page for my own example test:**
- [Mission/Angelus League Clusters #2 (2025)](https://ca.milesplit.com/meets/711006-missionangelus-league-clusters-2-2025/results)

---

### Step 2: Identify the Differences  

When comparing these pages, notice *how the data is structured differently*.  
Ask yourself questions like:
- Does my meet’s data appear inside a **table**, or is it just text?  
- Are the **column headers** (e.g., “Place”, “Team”, “Mark”) the same or different? 
- Do I see any unique features **in or around** the table?   

Here’s what I found in my own example:
1. Some meet results are formatted as plain text, not tables.   
2. My column headers are slightly different from the other formats.  
3. My meet results include **URLs inside the table** (for athletes and teams) — others do not. 

---

### Step 3: Inspect the HTML  

To confirm your visual observations:
1. Open the webpage in Chrome.  
2. Right-click on part of the results → select **“Inspect”**.  
3. Look closely at the HTML structure — especially the table tags (`<table>`, `<thead>`, `<tbody>`, `<th>`, `<td>`).  
4. Identify patterns that are *unique* to your page type (like a specific `class` name or missing headers).

This step is where you’ll find the **exact clues** your code can look for.

---

### Step 4: Write Logic to Capture Those Differences  

Now that you know what’s unique about your HTML, write Python logic that **detects** those differences.  
Your goal is to have a detector function that looks at a webpage and scores how likely it is to match your format (from `0.0` = definitely not yours to `1.0` = definitely yours).  

For example:
- +0.6 if the page has a table with the class `"eventTable"`.  
- +0.3 if the table headers match your expected names.  
- +0.1 if there are links inside the table body.

Notice that my sub-scores add up to 1.

Your function will combine these checks to produce a final confidence score between 0 and 1.

---

### Step 5: Understand the Expected Output  

Below is an example description of what your detector function should do conceptually:

> The function looks for features that are **unique** to my HTML file compared to my teammates’ files.  
> Each unique feature adds to a total score between 0 and 1.  
> The higher the score, the more confident the script is that this HTML belongs to my format.

Once you’ve implemented your version, you’ll test it on all the HTML samples to verify that:
- Your own HTML file scores close to **1.0**  
- Your teammates’ HTML files score close to **0.0**


#### Let's look at some of my code

Here is what my code looked like AFTER development, but before refactoring.  For the rest of this notebook we will work to replicate my process with your own function.

For now, let's get an overview of what I wrote for my detect_katie function.

In [ ]:
REQUIRED_HEADERS_KATIE = {"place", "video", "athlete", "team", "mark", "points"}

def detect_katie(html: str) -> float:
    """
    Return a confidence in [0,1] that this HTML matches the 'Katie' format. 
    NOTES FOR IMPLEMENTATION: 
    - Name your function detect_YOURNAME 
    - adjust signatures and weights for your format as needed 
    - you do not need to include all types of signals shown here if you can generate a reliable score with fewer 
    - be careful not to use signals that are too general and might appear in other formats 
    - aim for a final score that is well-calibrated (e.g., real Katie files score near 1.0, non-Katie files near 0.0) 
    
    Signals: 
    1) Table has class 'eventTable' (strong, +0.6) 
    2) Header names include Place/Video/Athlete/Team/Mark/Points (secondary, up to +0.3) 
    3) There are links (<a>) inside the table body (weak, +0.1)
    """
    soup = BeautifulSoup(html, "html.parser")
    score = 0.0

    # --- 1) Strong: find a table whose class list includes 'eventTable' (robust to spacing/case) ---
    event_table = None
    for tbl in soup.find_all("table"):
        cls = tbl.get("class", [])
        # BeautifulSoup usually gives a list; if string, split on whitespace
        if isinstance(cls, str):
            cls = cls.split()
        tokens = [c.strip().lower() for c in cls if c and c.strip()]
        if any("eventtable" == tok or "eventtable" in tok for tok in tokens):
            event_table = tbl
            break

    # If no EventTable found, score remains 0 and we exit early
    if event_table is None:
        return 0.0

    # Found the table → add strong credit
    score += 0.6

    # --- 2) Secondary: header names present (ignore blank headers like <th></th>) ---
    th_texts = [
        th.get_text(" ", strip=True).strip().lower()
        for th in event_table.select("thead th")
        if th.get_text(strip=True)
    ]
    headers_found = set(th_texts)
    matched = len(REQUIRED_HEADERS_KATIE.intersection(headers_found))
    if matched:
        score += 0.3 * (matched / len(REQUIRED_HEADERS_KATIE))  # proportional credit

    # --- 3) Weak: presence of links in tbody (athlete/team profile links) ---
    links_in_body = event_table.select("tbody a[href]")
    if len(links_in_body) >= 1:
        score += 0.05
    if len(links_in_body) >= 2:
        score += 0.05  # full weak credit

    return min(score, 1.0)

score = detect_katie(html) 
print(f"Katie format confidence score: {score:.2f}")


Katie format confidence score: 1.00


#### Development part 1: Write a function outline with the correct name and logic steps.

Let's create a roadmap by including the logic we expect to see as notes to ourself during this process.

In [ ]:
def detect_katie(html: str) -> float: ## replace the function name with your own
    """
    Return a confidence in [0,1] that this HTML matches the 'Katie' format. 
    NOTES FOR IMPLEMENTATION: 
    - Name your function detect_YOURNAME 
    - adjust signatures and weights for your format as needed 
    - you do not need to include all types of signals shown here if you can generate a reliable score with fewer 
    - be careful not to use signals that are too general and might appear in other formats 
    - aim for a final score that is well-calibrated (e.g., real Katie files score near 1.0, non-Katie files near 0.0) 
    
    Signals: 
    1) Table has class 'eventTable' (strong, +0.6) 
    2) Header names include Place/Video/Athlete/Team/Mark/Points (secondary, up to +0.3) 
    3) There are links (<a>) inside the table body (weak, +0.1)
    """
    ## Read in my html file with beautiful soup.

    # --- 1) Strong: find a table whose class list includes 'eventTable'

    # If no EventTable found, score remains 0 and we exit early

    # --- 2) Secondary: header names present (ignore blank headers like <th></th>) ---

    # --- 3) Weak: presence of links in tbody (athlete/team profile links) ---

    return min(score, 1.0) #return the final score, or 1 if the score is bigger than 1.

score = detect_katie(html) 
print(f"Katie format confidence score: {score:.2f}")

#### Developoment Part 2: Generate working logic for a distinguishing feature of your code.

Let's take the first distinguishing feature - the table name - and write code isolate that feature.

In [5]:
# --- 1) Strong: find a table whose class list includes 'eventTable' (robust to spacing/case) ---
event_table = None
for tbl in soup.find_all("table"):
    cls = tbl.get("class", [])
    # BeautifulSoup usually gives a list; if string, split on whitespace
    if isinstance(cls, str):
        cls = cls.split()
    tokens = [c.strip().lower() for c in cls if c and c.strip()]
    if any("eventtable" == tok or "eventtable" in tok for tok in tokens):
        event_table = tbl

event_table

<table class="eventTable" id="ts-1">
<thead>
<tr class="eventHeadRow">
<th>Place</th>
<th>Team</th>
<th>Points</th>
<th>Scorers</th>
<th>Times</th>
</tr>
</thead>
<tbody>
<tr>
<td class="place">
                                    1
                                </td>
<td class="tsTeam">
<img alt="Flintridge Sacred Heart (SS)" class="teamLogo" src="https://assets.sp.milesplit.com/team_logos/17018?v=1761233772"/>
<a href="https://ca.milesplit.com/teams/17018-flintridge-sacred-heart-ss">Flintridge Sacred Heart (SS)</a>
</td>
<td class="point">51</td>
<td class="wind">4+5+12+14+16  (17+20)</td>
<td class="heat">1:14 1-5 Split | 21:24 Avg</td>
</tr>
<tr>
<td class="place">
                                    2
                                </td>
<td class="tsTeam">
<img alt="Harvard Westlake (SS)" class="teamLogo" src="https://assets.sp.milesplit.com/team_logos/17103?v=1761233772"/>
<a href="https://ca.milesplit.com/teams/17103-harvard-westlake-ss">Harvard Westlake (SS)</a>
</td>
<td c

#### Development Part 2: Assign a score to your logic. 

In my function I want to assign a score to the table existing.  Since the table name is different between my html file and all other html files, I'm going to give this a "strong" score indicator.  Since some html files don't even have an event table, we will exit my detector if no table with the correct name is found.

In [ ]:
score=0.0
# If no EventTable found, score remains 0 and we exit early
if event_table is None:
    score = 0.0 #in the function we would exit here

# Found the table → add strong credit
score += 0.6

score

0.6

### Development Part 3: Do the same thing for other distinguishing features.

If the table exists, then we apply our weaker logic (urls existing in the table, the table names being correct) to verify that we should keep using my logic. Remember, in the end only the highest score ABOVE 0.8 will determine which web scraping algorithm to use.  So your detect code should return a score above 0.8 for your html, and below 0.3 for the other samples.

Continue writing logic for other differences you noticed between tables. Put it together into a function that takes an HTML file path as input and returns a score between 0 and 1 as a float.

Finally, test your function on all provided HTML files to ensure it correctly identifies your format with high confidence and others with low confidence.

In [ ]:
#Develop your other logic here

#### Development Part 4: Combine each of your indicators into the same function.  Make sure your score adds to at least 1. 

Below I've added print statements to help me test that my logic is working for each feature, and to track how the score changes.

In [ ]:
from bs4 import BeautifulSoup, Tag
from typing import Optional, Iterable, Set

# --- Configuration ---
REQUIRED_HEADERS_ADAM = {"place", "athlete", "grade", "school", "time"}

# --- Weights ---
W_STRONG = 0.65
W_HEADERS = 0.3
W_TABLECOUNT = 0.05
W_STRUCTURE = 0.20   # NEW bonus for Milesplit results header structure


def _normalize_tokens(tokens: Iterable[str]) -> Set[str]:
    return {t.strip().lower() for t in tokens if t and t.strip()}


def has_milesplit_results_header_structure(soup: BeautifulSoup) -> bool:
    article = soup.find("article")
    if not article:
        return False
    header = article.find("header")
    if not header:
        return False
    form = header.find("form", id="frmMeetResultsDetailFilter")
    if not form:
        return False
    select = form.find("select", id="ddResultsPage")
    if not select:
        return False
    if not select.find_all("option"):
        return False

    return True


def _find_meetresults_tables(soup: BeautifulSoup) -> list[Tag]:
    container = soup.find(id="meetResultsBody") or soup.find("div", id="meetResultsBody")
    if not container:
        return []
    return container.find_all("table")


# --- Main detector ---
def detect_adam(html: str) -> float:
    soup = BeautifulSoup(html, "html.parser")
    score = 0.0

    if has_milesplit_results_header_structure(soup):
        score += W_STRUCTURE

    tables = _find_meetresults_tables(soup)
    if not tables:
        return score  # no tables → return whatever structural score we have

    # Strong match for meetResultsBody structure
    score += W_STRONG

    # Header match (use best-scoring table)
    best_header_score = 0.0
    for tbl in tables:
        headers = _normalize_tokens(
            th.get_text(" ", strip=True)
            for th in tbl.find_all(["th", "td"])
        )
        overlap = len(REQUIRED_HEADERS_ADAM.intersection(headers))
        if overlap:
            best_header_score = max(best_header_score, overlap / len(REQUIRED_HEADERS_ADAM))

    score += W_HEADERS * best_header_score

    if len(tables) >= 2:
        score += W_TABLECOUNT

    return min(score, 1.0)


#### Development part 5: Refactor your function and ensure it still works properly.

Finally, ask your LLM to refactor the code you have developed with all parts included.  Ensure that the input is an html file path and the output is a float. The refactored code should be more easy for you to use in the future, or a teammate to use.

In [36]:
from bs4 import BeautifulSoup, Tag
from typing import Optional, Iterable, Set

# --- Configuration ---
REQUIRED_HEADERS_ADAM = {"place", "athlete", "grade", "school", "time"}

# --- Weights ---
W_STRONG = 0.65
W_HEADERS = 0.3
W_TABLECOUNT = 0.05
W_STRUCTURE = 0.20   # NEW bonus for Milesplit results header structure


def _normalize_tokens(tokens: Iterable[str]) -> Set[str]:
    return {t.strip().lower() for t in tokens if t and t.strip()}


def has_milesplit_results_header_structure(soup: BeautifulSoup) -> bool:
    article = soup.find("article")
    if not article:
        return False
    header = article.find("header")
    if not header:
        return False
    form = header.find("form", id="frmMeetResultsDetailFilter")
    if not form:
        return False
    select = form.find("select", id="ddResultsPage")
    if not select:
        return False
    if not select.find_all("option"):
        return False

    return True


def _find_meetresults_tables(soup: BeautifulSoup) -> list[Tag]:
    container = soup.find(id="meetResultsBody") or soup.find("div", id="meetResultsBody")
    if not container:
        return []
    return container.find_all("table")


# --- Main detector ---
def detect_adam(html: str) -> float:
    soup = BeautifulSoup(html, "html.parser")
    score = 0.0

    if has_milesplit_results_header_structure(soup):
        score += W_STRUCTURE

    tables = _find_meetresults_tables(soup)
    if not tables:
        return score  # no tables → return whatever structural score we have

    # Strong match for meetResultsBody structure
    score += W_STRONG

    # Header match (use best-scoring table)
    best_header_score = 0.0
    for tbl in tables:
        headers = _normalize_tokens(
            th.get_text(" ", strip=True)
            for th in tbl.find_all(["th", "td"])
        )
        overlap = len(REQUIRED_HEADERS_ADAM.intersection(headers))
        if overlap:
            best_header_score = max(best_header_score, overlap / len(REQUIRED_HEADERS_ADAM))

    score += W_HEADERS * best_header_score

    if len(tables) >= 2:
        score += W_TABLECOUNT

    return min(score, 1.0)


#### Development Part 6: Double check that your code correctly scores all htmls.

Verify that your code not only scores your html highly, but also gives everyone else's html a low score.  If your score is below 0.8 or a teammates score is above 0.5 then edit your function.

In [28]:
#Change this base path to match y"C:\Users\abw13\Research\research assistants copy\research assistants copy\raw html_files\\"our directory.
base_path = r'C:\Users\abw13\Research\research_assistants copy\research_assistants copy\raw_html_files\\'

In [37]:
import os

test_files_names = ['meet_44115.html', 'meet_44115.html', 'meet_493916.html', 'meet_494231.html']
test_files = [os.path.join(base_path, name) for name in test_files_names]

for name, file in zip(test_files_names, test_files):
    if not os.path.exists(file):
        print(f"❌ Missing file: {file}")
        continue
    with open(file, 'r', encoding='utf-8') as f:
        html = f.read()
    score = detect_adam(html)
    print(f"✅ File: {name}, Adam format confidence score: {score:.2f}")



✅ File: meet_44115.html, Adam format confidence score: 0.00
✅ File: meet_44115.html, Adam format confidence score: 0.00
✅ File: meet_493916.html, Adam format confidence score: 1.00
✅ File: meet_494231.html, Adam format confidence score: 0.20
